In [16]:
from scipy import constants

from PhysicalUnits import PhysicalUnits

tip_radius = 100.0*constants.nano
freq = 50.0*constants.tera
length_FD = 2.0*tip_radius
wp_si = 2.0*np.pi*1000.0*constants.tera

units = PhysicalUnits(length_FD)
freq_FD = units.ConvertSIFrequencyToFDUnits(freq)
tip_radius_FD = units.ConvertSILengthToFDUnits(tip_radius)
lambda_si = constants.c/freq
lambda_FD = 1.0/freq_FD
wp_FD = units.ConvertSIFrequencyToFDUnits(wp_si)

print("tip_radius_FD :", tip_radius_FD)
print("freq_FD: ", freq_FD)
print("lambda_si: ", lambda_si)
print("wp_FD: ", wp_FD)

tip_radius_FD : 0.5
freq_FD:  0.03335640951981521
lambda_si:  5.99584916e-06
wp_FD:  4.191690043903364


In [17]:
import numpy as np

from GridsParameters import MultilevelGridsParameters

box_0_scale = 8.0*tip_radius_FD

tip_height = 0.5*lambda_FD
print("tip_height (lambda) : ", tip_height/lambda_FD)

box_0_y0 = -tip_height/box_0_scale - 1.0
print("box_0_y0 : ", box_0_y0)

#box_0_y0 = -3.5

mlGrid = MultilevelGridsParameters()
mlGrid.SetCenterGridDimensions(r0 = np.array([-1.0, box_0_y0, -1.0])*box_0_scale,
                               r1 = np.array([+1.0, +1.0, +1.0])*box_0_scale,
                               dr = np.array([1.0, 1.0, 1.0])*0.2*tip_radius_FD,
                               S  = 0.9,
                               nFactor = 8,
                               nIsPowerOf2=False)

mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
#mlGrid.AddLayer(numOfCells = np.array([32, 32, 32]))
add_pml = False
if add_pml:
    mlGrid.AddPML(face = "f", numOfCells = 20)
    mlGrid.AddPML(face = "b", numOfCells = 20)
    mlGrid.AddPML(face = "r", numOfCells = 20)
    mlGrid.AddPML(face = "l", numOfCells = 20)

boxes = mlGrid.boxes
#print("boxes: ", boxes)
    
sourceType = "tip"
if sourceType == "pointSource":
    mlGrid.AddSource({"type": "GaussianPointSource", "position":np.array([0.0, 0.0, 0.0]), 
                      "polarization":"y", "amplitude":1.0, "t_center":1.0/freq_FD, "t_decay":0.25/freq_FD,
                      "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 , "timeOffsetFraction":0.5
                     })
elif sourceType == "lineSource":
    mlGrid.AddSource({"type": "GaussianLineSource_y", "position":np.array([0.0, 0.0, 0.0]), 
                      "height":100*box_0_scale,
                      "polarization":"y", "amplitude":1.0, "t_center":1.0/freq_FD, "t_decay":0.25/freq_FD,
                      "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 , "timeOffsetFraction":0.5
                     })
elif sourceType == "sheetSource":
    mlGrid.AddSource({"type": "GaussianSheetSource_z", "position":np.array([0.0, 0.0, -2.8])*box_0_scale, 
                      "x_width":15*box_0_scale, "y_width":100*box_0_scale,
                      "polarization":"y", "amplitude":1.0, "t_center":1.0/freq_FD, "t_decay":0.25/freq_FD,
                      "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 , "timeOffsetFraction":0.5
                     })
elif sourceType == "tip":
    mlGrid.AddGeometry({"type":"cone", "geometryName":"tip",
                        "coneAngle":1.0*(np.pi/180), 
                        "tipRadius":tip_radius_FD, "apexPosition":np.array([0.0, 0.0, 0.0])*box_0_scale,
                        "height":tip_height
                      })
    
    add_cylinders = False
    if add_cylinders:
        y_0_tip = -tip_height
        n_lev = len(boxes)
        y_prev = y_0_tip
        r_prev = tip_radius_FD
        for lev in range(1, n_lev):
            y_next = 0.5*(boxes[lev]["r0"][1] + boxes[lev - 1]["r0"][1])
            mlGrid.AddGeometry({"type":"cylinder", "geometryName":"cyl_" + str(lev),
                                "radius":r_prev, "topCenter":np.array([0.0, y_prev, 0.0]),
                                "height":y_prev - y_next
                              })
            y_prev = y_next
            r_prev *= 2.0
            
        mlGrid.AddGeometry({"type":"cylinder", "geometryName":"cyl_" + str(n_lev),
                            "radius":r_prev, "topCenter":np.array([0.0, y_prev, 0.0]),
                            "height": y_prev - boxes[-1]["r0"][1]
                          })
    use_pec = False
    pec_type = "pec_PureScattered"     ## "pec" / "pec_PureScattered"
    if use_pec:
        mlGrid.AddMaterial({"type":pec_type, "geometryName":"tip"})
        if add_cylinders:
            for lev in range(1, n_lev + 1):
                mlGrid.AddMaterial({"type":pec_type, "geometryName":"cyl_" + str(lev)})
    else:
        wp = wp_FD
        gamma = 2.0*np.pi*freq_FD
        mlGrid.AddMaterial({"type":"DrudeMetal_PureScattered", "geometryName":"tip",
                            "plasmaFrequency":wp, "scatteringRate":gamma})
        if add_cylinders:
            wp2_decrease_factor = 1.5
            for lev in range(1, n_lev + 1):
                mlGrid.AddMaterial({"type":"DrudeMetal_PureScattered", "geometryName":"cyl_" + str(lev),
                                    "wireMeshAlong":"y",
                                    "plasmaFrequency":wp/np.sqrt(lev*wp2_decrease_factor), 
                                    "scatteringRate":gamma})
        
    amp = 1.0
    prop_dir = np.array([0,0,1.0])
    prop_vel = 1.0
    t_center = 0.6/freq_FD
    rect_width = 1.0/freq_FD
    rectEdgeWidth = 0.1/freq_FD
    
    if pec_type == "pec":
        mlGrid.AddSource({"type": "GaussianSheetSource_z", "position":np.array([0.0, 0.0, -2.8])*box_0_scale, 
                          "x_width":15*box_0_scale, "y_width":100*box_0_scale,
                          "polarization":"y", "amplitude":1.0, "t_center":1.0/freq_FD, "t_decay":0.25/freq_FD,
                          "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 , "timeOffsetFraction":0.5
                         })
    elif pec_type == "pec_PureScattered":
        mlGrid.AddSource({"type": "PureScatteredRectPlaneWave", "geometryName":"tip",
                          "polarization":"y", "amplitude":amp, 
                          "propagationDirection":prop_dir, "propagationVelocity":prop_vel,
                          "t_center":t_center, "rectWidth":rect_width, "rectEdgeWidth":rectEdgeWidth,
                          "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 
                     })
        if add_cylinders:
            for lev in range(1, n_lev + 1):
                mlGrid.AddSource({"type": "PureScatteredRectPlaneWave", "geometryName":"cyl_" + str(lev),
                                  "polarization":"y", "amplitude":amp, 
                                  "propagationDirection":prop_dir, "propagationVelocity":prop_vel,
                                  "t_center":t_center, "rectWidth":rect_width, "rectEdgeWidth":rectEdgeWidth,
                                  "modulationFrequency":freq_FD , "modulationPhase":np.pi/2 
                             })


mlGrid.AddView({"type":"partial", "plane":"x", "at":0.0, "direction":"y", "arrayName":"E"})
mlGrid.AddView({"type":"partial", "plane":"y", "at":0.0, "direction":"y", "arrayName":"E"})
mlGrid.AddView({"type":"partial", "plane":"z", "at":0.0, "direction":"y", "arrayName":"E"})

y_test = boxes[-2]["r0"][1] + 2*boxes[-2]["dr"][2]
mlGrid.AddView({"type":"partial", "plane":"y", "at":y_test, "direction":"y", "arrayName":"E"})
y_test = boxes[-2]["r0"][1] - 4*boxes[-2]["dr"][2]
mlGrid.AddView({"type":"partial", "plane":"y", "at":y_test, "direction":"y", "arrayName":"E"})


paramfileName = "../../../data/3D/auto/params.param"
mlGrid.SetupCollectionAndRun(t_max = 1.2/freq_FD, 
                             filename = "../../processed/Maxwell3D_nonuniform_autogenerated.json", 
                             paramfileName = paramfileName
                             )

import pickle
fileParams = open(paramfileName, "rb")
params = pickle.load(fileParams)
fileParams.close()

params["units"] = {"tip_radius":tip_radius, "freq":freq, "length_FD":length_FD, "sourceType":sourceType}

paramfile = open(paramfileName, "wb")
pickle.dump(params, paramfile)
paramfile.close()

(params["boxes"][-1]["r1"] - params["boxes"][-1]["r0"])*length_FD/lambda_si

tip_height (lambda) :  0.5
box_0_y0 :  -4.747405725
Num of time steps:  92
gridsIntersectingGeometries  {'grid_m0': {'tip': [array([ -0.62646806, -14.9896229 ,  -0.62646806]), array([0.62646806, 0.        , 0.62646806])]}, 'grid_r1': {}, 'grid_l1': {}, 'grid_u1': {}, 'grid_d1': {}, 'grid_f1': {}, 'grid_b1': {}, 'grid_r2': {}, 'grid_l2': {}, 'grid_u2': {}, 'grid_d2': {}, 'grid_f2': {}, 'grid_b2': {}, 'grid_r3': {}, 'grid_l3': {}, 'grid_u3': {}, 'grid_d3': {}, 'grid_f3': {}, 'grid_b3': {}}


array([3.29925214, 4.01935841, 3.29925214])